In [2]:
import math
from collections import Counter, defaultdict
from docx import Document
import os
import json
import random

In [3]:
def read_docx(file_path):
    """读取DOCX文件并返回其中的文本内容"""
    doc = Document(file_path)
    text = []
    for para in doc.paragraphs:
        text.append(para.text)
    return ''.join(text)

def collect_statistics(text):
    """统计汉字、标点符号和空格的频率"""
    char_count = Counter(text)
    total_chars = sum(char_count.values())
    return char_count, total_chars

In [4]:
def calculate_entropy(filtered_text, order=0):
    if order == 0:
        # 0阶马尔可夫模型：每个字符独立选择
        char_count = Counter(filtered_text)
        total_chars = sum(char_count.values())
        entropy = 0
        for count in char_count.values():
            probability = count / total_chars
            entropy -= probability * math.log2(probability)
        return entropy
    else:
        # 高阶马尔可夫模型
        storage = defaultdict(Counter)
        total_cal = len(filtered_text) - order

        for i in range(total_cal):
            prefix = filtered_text[i:i + order]
            next_char = filtered_text[i + order]
            storage[prefix][next_char] += 1

        entropy = 0
        for prefix, suffix_counts in storage.items():
            prefix_total = sum(suffix_counts.values())
            for count in suffix_counts.values():
                probability = count / prefix_total
                entropy -= (prefix_total / total_cal) * probability * math.log2(probability)

        return entropy

In [5]:
def markov_model(text, order):
    """构建马尔可夫模型并输出模型状态和转移概率"""
    filtered_text = text
    storage = defaultdict(Counter)

    for i in range(len(filtered_text) - order):
        prefix = filtered_text[i:i + order]
        next_char = filtered_text[i + order]
        storage[prefix][next_char] += 1

    total_storage = sum(sum(suffix_counts.values()) for suffix_counts in storage.values())
    model = {}
    for prefix, suffix_counts in storage.items():
        prefix_total = sum(suffix_counts.values())
        model[prefix] = {char: count / prefix_total for char, count in suffix_counts.items()}

    return model, total_storage

In [6]:
def print_markov_model(model):
    """打印马尔可夫模型的状态和转移概率"""
    for prefix, transitions in model.items():
        print(f"State: {prefix}")
        for char, prob in transitions.items():
            print(f"  {char}: {prob:.4f}")

由于数据过少，导致只有零阶熵的数据比较准确。
参考文献：
在国内最早冯志伟先生用了将近 10 年的时间，进行手工查频，从小到大地逐步扩大统计的规模，建立了 6 个不同容量的汉字频度表，最后根据这些不同的汉字频度表，逐步地扩大汉字的容量，终于在 70 年代末期首次计算出了在不考虑上下文影响的前提下汉字信息熵的值是 9.65 比特……黄萱菁等在 4 年的《人民日报》语料的基础上，所求得的零阶熵、一阶熵、二阶熵分别为 9.62，6.18 和 4.89 比特。 刘源给出汉字熵的计算结果是9.71 比特。 孙帆等基于词的语言模型估计方法比基于字的直接计算方法得到了汉字熵的更为精确的估计，其熵值为 5.31 比特。

In [7]:
Text_file = 'Zhengfei_Ren_Email.docx'
Text = read_docx(Text_file)

Text_stats, elon_total = collect_statistics(Text)
print("Zhengfei_Ren_Email Statistics:")
print(Text_stats)

Zhengfei_Ren_Email Statistics:
Counter({'，': 164, '的': 83, '。': 57, '是': 52, '一': 51, '人': 42, '才': 36, '们': 34, '就': 34, '有': 33, '我': 32, '要': 31, '他': 31, '在': 29, '不': 29, '、': 24, '个': 21, '专': 21, '家': 21, '工': 19, '来': 19, '成': 18, '大': 18, '界': 17, '能': 17, '高': 15, '时': 15, '以': 15, '定': 15, '如': 15, '学': 15, '到': 15, '为': 14, '上': 14, '也': 14, '自': 13, '去': 13, '多': 13, '会': 12, '可': 12, '这': 12, '了': 12, '业': 11, '边': 11, '力': 11, '没': 11, '作': 10, '么': 10, '理': 10, '很': 10, '长': 10, '级': 10, '“': 10, '”': 10, '…': 10, '华': 9, '司': 8, '先': 8, '全': 8, '产': 8, '员': 8, '果': 8, '主': 8, '生': 8, '实': 8, '件': 8, '己': 7, '优': 7, '秀': 7, '清': 7, '公': 7, '方': 7, '所': 7, '当': 7, '什': 7, '愿': 7, '意': 7, '中': 7, '出': 7, '然': 7, '部': 7, '合': 7, '小': 7, '做': 7, '端': 6, '对': 6, '0': 6, '储': 6, '备': 6, '务': 6, '内': 6, '经': 6, '位': 6, '事': 6, '行': 6, '比': 6, '只': 6, '面': 6, '让': 6, '找': 6, '献': 6, '价': 6, '和': 6, '新': 6, '题': 6, '咖': 6, '啡': 6, '开': 6, '平': 6, '创': 5, '任': 5, '术': 5, '2': 5, 

In [8]:
Text_entropy_0 = calculate_entropy(Text, 0)
Text_entropy_3 = calculate_entropy(Text, 3)
Text_entropy_5 = calculate_entropy(Text, 5)
print(f"\nText Entropy (0th order): {Text_entropy_0}")
print(f"Text Entropy (3rd order): {Text_entropy_3}")
print(f"Text Entropy (5th order): {Text_entropy_5}")


Text Entropy (0th order): 8.11198715230113
Text Entropy (3rd order): 0.14993218305713354
Text Entropy (5th order): 0.017173222905156053


In [9]:
orders = [0, 3, 5]
for order in orders:
    print(f"\nZhengfei_Ren_Email Markov Model (Order {order}):")
    elon_model, elon_total_ngrams = markov_model(Text, order)
    print_markov_model(elon_model)

    with open(f'Zhengfei_Ren_Email Markov Model (Order {order}).txt', 'w', encoding='utf-8',) as f:
        f.write(str(elon_model))
        f.close()


Zhengfei_Ren_Email Markov Model (Order 0):
State: 
  华: 0.0033
  为: 0.0052
  创: 0.0019
  始: 0.0007
  人: 0.0156
  任: 0.0019
  正: 0.0004
  非: 0.0011
  在: 0.0107
  高: 0.0056
  端: 0.0022
  技: 0.0015
  术: 0.0019
  才: 0.0133
  使: 0.0007
  用: 0.0015
  工: 0.0070
  作: 0.0037
  组: 0.0004
  对: 0.0022
  标: 0.0007
  会: 0.0044
  上: 0.0052
  的: 0.0308
  讲: 0.0015
  话: 0.0007
  2: 0.0019
  0: 0.0022
  3: 0.0004
  年: 0.0015
  7: 0.0004
  月: 0.0004
  8: 0.0004
  日: 0.0004
  一: 0.0189
  、: 0.0089
  我: 0.0119
  们: 0.0126
  要: 0.0115
  建: 0.0015
  立: 0.0011
  个: 0.0078
  自: 0.0048
  己: 0.0026
  储: 0.0022
  备: 0.0022
  库: 0.0011
  ，: 0.0608
  不: 0.0107
  拘: 0.0007
  格: 0.0015
  获: 0.0007
  取: 0.0007
  优: 0.0026
  秀: 0.0026
  但: 0.0011
  招: 0.0019
  聘: 0.0015
  时: 0.0056
  清: 0.0026
  楚: 0.0011
  公: 0.0026
  司: 0.0030
  业: 0.0041
  务: 0.0022
  边: 0.0041
  界: 0.0063
  允: 0.0004
  许: 0.0007
  内: 0.0022
  研: 0.0015
  究: 0.0011
  探: 0.0007
  索: 0.0011
  。: 0.0211
  已: 0.0004
  经: 0.0022
  明: 0.0007
  确: 0.0019


我在Github上找到了2015-2019年每天新闻联播的语料，数据的数量级在千万级别，喂给模型计算。得到的结果更接近于文献中的数据。

In [10]:
folder_path = 'news_content/'

def read_files(folder_path):
    texts = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.json'):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                data = json.load(file)
                texts.append(data)
    return ''.join(str(texts))

text1 = read_files(folder_path)
# 过滤非字母和非空格字符，并转换为小写
filtered_text1 = ''.join(filter(lambda x: '\u4e00' <= x <= '\u9fff' or x.isspace(), text1))
char_count1 = Counter(filtered_text1)

print(char_count1)

Counter({'的': 301786, '国': 242667, '中': 181505, '在': 113234, '一': 111648, '会': 110464, '人': 108764, '和': 103585, '新': 103106, '大': 101718, '发': 99878, '作': 84672, '全': 79576, '年': 78988, '了': 73124, '展': 69458, '要': 67384, '为': 63928, ' ': 63465, '民': 62935, '方': 62718, '进': 61914, '行': 61624, '出': 61612, '央': 60098, '主': 59886, '平': 59224, '合': 58626, '是': 58562, '政': 57646, '地': 57066, '家': 55278, '联': 54074, '有': 53956, '同': 53884, '动': 53614, '成': 53042, '实': 52986, '业': 52970, '文': 52888, '对': 51406, '日': 51268, '部': 50876, '强': 50726, '关': 50604, '开': 50468, '上': 49824, '近': 49478, '建': 48444, '共': 47103, '重': 46512, '工': 46496, '加': 46236, '不': 44622, '经': 44402, '习': 44304, '力': 43314, '时': 42394, '来': 42062, '党': 42052, '表': 41878, '总': 41834, '生': 41516, '多': 41468, '议': 41414, '视': 41164, '以': 41118, '高': 40774, '个': 40621, '化': 40406, '播': 39514, '天': 39140, '务': 38460, '持': 38366, '等': 37760, '网': 37672, '员': 37638, '法': 37485, '区': 37260, '到': 37014, '长': 36592, '体': 3567

In [11]:
orders = [0, 3, 5]
for order in orders:
    Chinese_Text_model, Chinese_Text_ngrams = markov_model(filtered_text1, order)

In [12]:
def generate_text(model, order, length=100):
    """使用马尔可夫模型随机生成文本"""
    generated_text = []
    prefix = random.choice(list(model.keys()))
    generated_text.extend(prefix)

    for _ in range(length - order):
        if prefix in model:
            next_char = random.choices(
                list(model[prefix].keys()),
                list(model[prefix].values())
            )[0]
            generated_text.append(next_char)
            prefix = prefix[1:] + next_char
        else:
            prefix = random.choice(list(model.keys()))
            generated_text.extend(prefix)

    return ''.join(generated_text[:length])

In [19]:
print(generate_text(Chinese_Text_model, 5, 200))

村党总支在搭建这个大数据平台他们目前已售出的多万台工程装备如今在什么位置是什么运行状态都一目了然现在医疗云覆盖到乡老百姓远程看病更方便工业云正用大数据改善政府服务满足群众需求决定建设福厦泉与合芜蚌两个国家自主创新示范区建设正式启动日前东盟与中日韩领导人会议日在泰国曼谷唐人街拉开帷幕泰国诗琳通公主亲自拉下了庆典启动闸门中泰两国的艺术家共同演唱的歌剧选段凯旋进行曲和今夜无人入睡中落下帷幕现场洋溢着欢乐
